<a href="https://colab.research.google.com/github/Vaycold/tensorflow_tutorial/blob/main/%2317.Keras%EB%A5%BC%20%ED%99%9C%EC%9A%A9%ED%95%9C%20%EB%B6%84%EC%82%B0%ED%98%95%20%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

    tf.distribute.Strategy API는 훈련을 여러 처리 장치들로 분산시키는 것을 추상화한 것입니다. 
    기존의 모델이나 훈련 코드를 조금만 바꾸어 분산 훈련을 할 수 있게 하는 것이 분산 전략 API의 목표입니다.

In [1]:
#!pip install -q tensorflow-gpu==2.1.0-rc1
import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os

In [2]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = datasets['train'], datasets['test']

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [3]:
# 분산 전략 정의하기
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [4]:
print('장치의 수: {}'.format(strategy.num_replicas_in_sync))

장치의 수: 1


In [5]:
# 정규화

def scale(image, label) :
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

In [6]:
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [7]:
BATCH_SIZE

64

In [8]:
train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [9]:
# Model 만들기

# strategy.scope() : 컨텍스트 안에서 케라스 모델 만들고 컴파일

with strategy.scope() :
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation = 'relu', input_shape=(28,28,1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer = tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])

In [10]:
os.listdir()

['.config', 'sample_data']

    콜백 정의하기
     - 텐서보드 : 이 콜백은 텐서보드용 로그를 남겨서 텐서보드에서 그래프를 그릴 수 있게 해줌
     - 모델 체크포인트 : 이 콜백은 매 에포크가 끝난 후 모델을 저장
     - 학습률 스케쥴러 : 매 에포크 혹은 배치가 끝난 후 학습률 변경가능
    

In [16]:
# 체크포인트를 저장할 체크포인트 디렉터리 지정
checkpoint_dir = './training_checkpoints'

#체크포인트 파일의 이름
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

In [13]:
checkpoint_prefix

'./training_checkpoints/ckpt_{epoch}'

In [14]:
# 학습률을 점점 줄이는 함수
def decay(epoch) :
    if epoch < 3 :
        return 1e-3
    elif epoch >= 3 and epoch < 7 :
        return 1e-4
    else :
        return 1e-5

In [15]:
# 에폭이 끝날 때마다 학습률을 출력하는 콜백
class PrintLR(tf.keras.callbacks.Callback) :
    def on_epoch_end(self,epoch, logs=None) :
        print('\n에포크 {}의 학습률은 {}입니다.'.format(epoch+1, model.optimizer.lr.numpy()))

In [18]:
callbacks=[
        tf.keras.callbacks.TensorBoard(log_dir ='./logs'),
        tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_prefix,
                                           save_weights_only=True),
        tf.keras.callbacks.LearningRateScheduler(decay),
        PrintLR()
]

In [19]:
train_dataset

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [20]:
model.fit(train_dataset, epochs=12, callbacks=callbacks)

Epoch 1/12
938/938 [==============================] - 35s 34ms/step - loss: 0.1977 - accuracy: 0.9430

에포크 1의 학습률은 0.0010000000474974513입니다.
Epoch 2/12
938/938 [==============================] - 29s 30ms/step - loss: 0.0665 - accuracy: 0.9798

에포크 2의 학습률은 0.0010000000474974513입니다.
Epoch 3/12
938/938 [==============================] - 29s 30ms/step - loss: 0.0463 - accuracy: 0.9859

에포크 3의 학습률은 0.0010000000474974513입니다.
Epoch 4/12
938/938 [==============================] - 28s 30ms/step - loss: 0.0247 - accuracy: 0.9931

에포크 4의 학습률은 9.999999747378752e-05입니다.
Epoch 5/12
938/938 [==============================] - 28s 29ms/step - loss: 0.0214 - accuracy: 0.9947

에포크 5의 학습률은 9.999999747378752e-05입니다.
Epoch 6/12
938/938 [==============================] - 28s 29ms/step - loss: 0.0196 - accuracy: 0.9948

에포크 6의 학습률은 9.999999747378752e-05입니다.
Epoch 7/12
938/938 [==============================] - 28s 29ms/step - loss: 0.0182 - accuracy: 0.9955

에포크 7의 학습률은 9.999999747378752e-05입니다.
Epoch 8/12
93

In [22]:
!ls {checkpoint_dir}

checkpoint		     ckpt_4.data-00000-of-00001
ckpt_10.data-00000-of-00001  ckpt_4.index
ckpt_10.index		     ckpt_5.data-00000-of-00001
ckpt_11.data-00000-of-00001  ckpt_5.index
ckpt_11.index		     ckpt_6.data-00000-of-00001
ckpt_12.data-00000-of-00001  ckpt_6.index
ckpt_12.index		     ckpt_7.data-00000-of-00001
ckpt_1.data-00000-of-00001   ckpt_7.index
ckpt_1.index		     ckpt_8.data-00000-of-00001
ckpt_2.data-00000-of-00001   ckpt_8.index
ckpt_2.index		     ckpt_9.data-00000-of-00001
ckpt_3.data-00000-of-00001   ckpt_9.index
ckpt_3.index
